In [36]:
# patterned after scraping code found here: https://github.com/jacobbaruch/NBA_data_scraping_and_analysis

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [13]:
import ast

# retrieve the available dates for boxscores
showdates_url = 'https://basketball.realgm.com/ajax/nba_scoreboard.phtml?action=showdates'
dates_resp = requests.get(showdates_url)
dates = ast.literal_eval(dates_resp.text)

In [ ]:
start_date = [...]
end_date = [...]

In [17]:
# for each date of interest, load e.g.:
#"https://basketball.realgm.com/nba/scores/{}/All".format("2018-11-01")

# within that page, find tables with class="game played"
# within those tables, find the a href with text of "Box Score"
# follow link to box score page

page_resp = requests.get("https://basketball.realgm.com/nba/scores/{}/All".format("2018-11-01"))

In [18]:
soup = BeautifulSoup(page_resp.text, 'html.parser')

In [28]:
[link.get('href') for link in soup.select('table.game.played a') if link.text == 'Box Score']

['/nba/boxscore/2018-11-01/LA-Clippers-at-Philadelphia/308916',
 '/nba/boxscore/2018-11-01/Denver-at-Cleveland/308915',
 '/nba/boxscore/2018-11-01/Oklahoma-City-at-Charlotte/308914',
 '/nba/boxscore/2018-11-01/Sacramento-at-Atlanta/308917',
 '/nba/boxscore/2018-11-01/Milwaukee-at-Boston/308918',
 '/nba/boxscore/2018-11-01/New-Orleans-at-Portland/308919']

In [2]:
#https://basketball.realgm.com/nba/boxscore/2018-11-01/LA-Clippers-at-Philadelphia/308916

page_resp = requests.get('https://basketball.realgm.com/nba/boxscore/2018-11-01/LA-Clippers-at-Philadelphia/308916')

In [3]:
soup = BeautifulSoup(page_resp.text, 'html.parser')

In [18]:
headers = soup.find_all("h2")
tables = soup.find_all('table')

In [19]:
headers[1]

<h2>Los Angeles Clippers</h2>

In [10]:
tables[3]

<table class="tablesaw compact" data-tablesaw-mode="swipe" data-tablesaw-mode-exclude="columntoggle" data-tablesaw-mode-switch="" data-tablesaw-sortable="" data-tablesaw-sortable-switch="">
<thead>
<tr>
<th>#</th>
<th data-tablesaw-priority="persist" data-tablesaw-sortable-col="" data-tablesaw-sortable-default-col="">Player</th>
<th data-tablesaw-sortable-col="">Status</th>
<th data-tablesaw-sortable-col="">Pos</th>
<th data-tablesaw-sortable-col="">Min</th>
<th data-tablesaw-sortable-col="">FGM-A</th>
<th data-tablesaw-sortable-col="">3PM-A</th>
<th data-tablesaw-sortable-col="">FTM-A</th>
<th data-tablesaw-sortable-col="" title="FIC: The Floor Impact Counter (FIC) used here is modified by RealGM's Christopher Reina to give more value to assists and blocks, differentiate between offensive and defensive rebounds (offensive being more valuable) and slightly reduces the value of missed field goals and free throws.">FIC</th>
<th data-tablesaw-sortable-col="">Off</th>
<th data-tablesaw-sor

In [33]:
players = []
for row in tables[3].find_all('tr')[1:]:
    cols = row.find_all('td')

    player = {}
    player['number'] = cols[0].string
    player['name'] = cols[1].string
    player['status'] = cols[2].string
    player['position'] = cols[3].string
    player['minutes'] = cols[4].string
    player['fgm-a'] = cols[5].string
    player['3pm-a'] = cols[6].string
    player['ftm-a'] = cols[7].string
    player['fic'] = cols[8].string
    player['off'] = cols[9].string
    player['def'] = cols[10].string
    player['reb'] = cols[11].string
    player['ast'] = cols[12].string
    player['pf'] = cols[13].string
    player['stl'] = cols[14].string
    player['blk'] = cols[15].string
    player['to'] = cols[16].string
    player['pts'] = cols[17].string
    
    players.append(player)

In [35]:
df = pd.DataFrame(players)
df.head()

,3pm-a,ast,blk,def,fgm-a,fic,ftm-a,minutes,name,number,off,pf,position,pts,reb,status,stl,to
0,0-4,4,0,2,1-7,1.8,2-4,30:23,Patrick Beverley,21,2.0,6,PG,4,4,Starter,2,2
1,0-1,1,0,1,1-6,-1.2,0-0,36:21,Avery Bradley,22,1.0,1,SG,2,2,Starter,1,2
2,4-8,1,0,8,5-14,15.9,11-11,35:28,Danilo Gallinari,8,0.0,1,PF,25,8,Starter,0,1
3,0-0,1,1,1,0-1,-1.0,0-0,09:08,Marcin Gortat,13,1.0,4,C,0,2,Starter,0,2
4,2-3,3,1,8,10-16,19.8,2-2,33:23,Tobias Harris,34,0.0,1,SF,24,8,Starter,0,1


In [20]:
headers[2]

<h2>Philadelphia Sixers</h2>

In [22]:
tables[4]

<table class="tablesaw compact" data-tablesaw-mode="swipe" data-tablesaw-mode-exclude="columntoggle" data-tablesaw-mode-switch="" data-tablesaw-sortable="" data-tablesaw-sortable-switch="">
<thead>
<tr>
<th>#</th>
<th data-tablesaw-priority="persist" data-tablesaw-sortable-col="" data-tablesaw-sortable-default-col="">Player</th>
<th data-tablesaw-sortable-col="">Status</th>
<th data-tablesaw-sortable-col="">Pos</th>
<th data-tablesaw-sortable-col="">Min</th>
<th data-tablesaw-sortable-col="">FGM-A</th>
<th data-tablesaw-sortable-col="">3PM-A</th>
<th data-tablesaw-sortable-col="">FTM-A</th>
<th data-tablesaw-sortable-col="" title="FIC: The Floor Impact Counter (FIC) used here is modified by RealGM's Christopher Reina to give more value to assists and blocks, differentiate between offensive and defensive rebounds (offensive being more valuable) and slightly reduces the value of missed field goals and free throws.">FIC</th>
<th data-tablesaw-sortable-col="">Off</th>
<th data-tablesaw-sor